# Generar vectorstore con toda la normativa ARN

In [16]:
# Determina entorno de ejecución: local o Colab

# if (firstrun):
if('google.colab' in str(get_ipython() ) ):
    environment= 'google'
else:
    import os
    if (os.environ.get('PWD')=='/kaggle/working'):
        environment= 'kaggle'
    else:
        environment= 'local'
print(environment)

local


In [ ]:
# Descarga de texto normas
if (environment=='google'):
  ! wget https://github.com/bettachini/nlpTP/blob/main/arn/gu%C3%ADas_texto.zip?raw=true -O guías_texto.zip
  ! unzip guías_texto.zip
  ! rm guías_texto.zip

  ! wget https://github.com/bettachini/nlpTP/blob/main/arn/normas_texto.zip?raw=true -O normas_texto.zip
  ! unzip normas_texto.zip
  ! rm normas_texto.zip

## 1. Load Text Files from Directory and Extract Document Identifier

To read multiple text files from a directory, load only the base names (ignoring any extra suffix), and assign them as document numbers.

In [1]:
import os
try: 
  from langchain_community.document_loaders import TextLoader
  from langchain.schema import Document  # Assuming this is the document class for LangChain
except ImportError:
  ! pip install langchain-community
  from langchain_community.document_loaders import TextLoader
  from langchain.schema import Document  # Assuming this is the document class for LangChain


In [2]:
# Normas

def text_loader_normas(directory_path, documents_list):
  for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
      # Extract the document number from the filename (e.g., 'a-b-c' from 'a-b-c_extra.txt')
      doc_number = 'AR ' + '.'.join(filename.split('_')[0].split('-'))
        
      # Load the file's content
      loader = TextLoader(os.path.join(directory_path, filename))
      doc_content = loader.load()

      # Append document with metadata for vector store
      for page in doc_content:
        documents_list.append(Document(page_content=page.page_content, metadata={"doc_number": doc_number}))

      #documents_list.append({
      #  "content": doc_content[0].page_content,
      #  "metadata": {"doc_number": doc_number}
      #})
  return documents_list

# Define the directory containing your text files
normas_path = './normas_texto'  # Replace with your directory path
normas_lista = []
normas_lista = text_loader_normas(normas_path, normas_lista)

In [3]:
# Guías

def text_loader_guías(directory_path, documents_list):
  for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
      # Extract the document number from the filename (e.g., 'a-b-c' from 'a-b-c_extra.txt')
      # doc_number = 'AR Guía' + '.'.join(filename.split('_')[0].split('-'))
      doc_number = 'Guía AR ' + filename.split('_')[0].split('-')[0][2:]
        
      # Load the file's content
      loader = TextLoader(os.path.join(directory_path, filename))
      doc_content = loader.load()
      
      
      # Append document with metadata for vector store
      for page in doc_content:
        documents_list.append(Document(page_content=page.page_content, metadata={"doc_number": doc_number}))

      #documents_list.append({
      #  "content": doc_content[0].page_content,
      #  "metadata": {"doc_number": doc_number}
      #})
  return documents_list

# same but for guías_texto
guías_path = './guías_texto'  # Replace with your directory path
guías_lista = []
guías_lista = text_loader_guías(guías_path, guías_lista)

In [4]:
normativa_lista = normas_lista + guías_lista

## 2. Split Documents into Chunks (if needed)

Since the code already uses a RecursiveCharacterTextSplitter, you can apply it to the documents list directly:

In [ ]:
documents = normativa_lista
# documents = normas_lista

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

## 3. Generate Embeddings and Store in Persistent Vector Store

To save the Chroma vector store to a file, specify a persistent storage path.

In [7]:
try:
  from langchain_ollama import OllamaEmbeddings
except ImportError:
  ! pip install langchain-ollama
  from langchain_ollama import OllamaEmbeddings

# Initialize the embedding model
embeddings_model = OllamaEmbeddings(model="all-minilm")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
try:
    import chromadb
except ImportError:
    ! pip install chromadb
from langchain.vectorstores import Chroma

In [ ]:
# Specify a persistent storage path for Chroma
vectorstore_path = './vectorstore'  # Replace with your path

# Create or load an existing Chroma vector store
try:
  vectorstore = Chroma.from_documents(
    documents=split_documents, 
    embedding=embeddings_model, 
    persist_directory=vectorstore_path
  )
except:
  #Installs Ollama
  !curl -fsSL https://ollama.com/install.sh | sh
  # Runs Ollama service in the background
  !ollama serve &>/dev/null&
  # Downloads the pre-trained model
  ! ollama pull all-minilm

  vectorstore = Chroma.from_documents(
    documents=split_documents, 
    embedding=embeddings_model, 
    persist_directory=vectorstore_path
  )
# Save the vector store to disk
vectorstore.persist()

ConnectError: [Errno 111] Connection refused

## 4. Loading the Persistent Vector Store

Later, to retrieve the vector store without regenerating embeddings, load it from the saved path:

In [ ]:
# Load the existing vector store
vectorstore = Chroma(persist_directory=vectorstore_path, embedding=embeddings_model)
